# **Week 13: Data Aggregation and Group Operation**
Please refer to the textbook: **"Python for Data Analysis"** by Wes McKinney for details of this topic on **Chapter 10**

# **Start by importing NumPy and pandas**

In [7]:
import numpy as np
import pandas as pd

# **13.1 Groupby Operations**
## The popular ***split-apply-combine***

In [121]:
# Example DataFrame
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({'key1' : ['a', 'a', None, 'b', 'b', 'a', None],
                   'key2' : pd.Series([1, 2, 1, 2, 1, None, 1], dtype='Int64'),
                   'data1' : rng.standard_normal(7),
                   'data2' : rng.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [123]:
# Access data1 and call groupby with the key1 column
# Create an intermediate file -> does not directly display the data
grouped = df['data1'].groupby(df['key1'])

In [125]:
# Method 1: We create explicitly the intermediate file first
# Compute median
# groupby create an intermediate result
# Where we can work on
%timeit grouped = df['data1'].groupby(df['key1']); grouped.median()

2.34 ms ± 211 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [126]:
# Method 2: We unwrap the intermediate file
# Summarize and analyze data by groups
%timeit df['data1'].groupby(df['key1']).median()

2.39 ms ± 400 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### **In actual case**....
The unwrap method is running faster

In [129]:
# Another example; passing multiple groupby arrays of keys to find mean 
# long format
means= df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     1      -1.423825
      2       1.263728
b     1      -0.075343
      2      -0.259173
Name: data1, dtype: float64

In [130]:
# Unstack -> wide format
means.unstack()

key2,1,2
key1,,
a,-1.423825,1.263728
b,-0.075343,-0.259173


In [131]:
# Display df content
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [132]:
# Pass column names as the group keys
df.groupby('key1').mean()

,key2,data1,data2
key1,,,
a,1.5,-0.300327,0.083521
b,1.5,-0.167258,1.657953


In [134]:
# Combination of keys
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    1    -1.423825  0.648893
     2     1.263728  0.361058
b    1    -0.075343  0.968497
     2    -0.259173  2.347410

In [137]:
# Returns a Series containing group sizes:
df.groupby(['key1', 'key2']).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

# Display content
df

In [142]:
# Missing values in a group key are excluded from the result by default
# Can be disabled by passing dropna=False to groupby
# dropna=True is by default
df.groupby(['key1','key2'], dropna=False).size()


key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

### **Iterating over Groups**
***groupby*** supports ***iteration***, generating ***a sequence of 2-tuples***

In [145]:
# Display df content
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [146]:
# Iteration
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -1.423825  0.648893
1    a     2  1.263728  0.361058
5    a  <NA> -0.740885 -0.759387
b
  key1  key2     data1     data2
3    b     2 -0.259173  2.347410
4    b     1 -0.075343  0.968497


In [147]:
# in the case of mu;tiple keys, first element in the tuple will be 
# a tuple of key values
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -1.423825  0.648893
('a', 2)
  key1  key2     data1     data2
1    a     2  1.263728  0.361058
('b', 1)
  key1  key2     data1     data2
4    b     1 -0.075343  0.968497
('b', 2)
  key1  key2     data1    data2
3    b     2 -0.259173  2.34741


In [149]:
# Computiing a dictionary using one-liner
# dictionary: key:value pair
pieces= {name: group for name, group in df.groupby('key1')}

In [150]:
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -1.423825  0.648893
 1    a     2  1.263728  0.361058
 5    a  <NA> -0.740885 -0.759387,
 'b':   key1  key2     data1     data2
 3    b     2 -0.259173  2.347410
 4    b     1 -0.075343  0.968497}

In [151]:
# Accessing item in dictionary
# Given the particular key -> 'a', or 'b'
pieces['a']

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
5,a,<NA>,-0.740885,-0.759387


In [152]:
# Acessing item in dictionary
pieces['b']

,key1,key2,data1,data2
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497


### **Selecting a Column or Subset of Columns**

In [154]:
# Returned a Dataframe if a list or array is passed
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     1       0.648893
      2       0.361058
b     1       0.968497
      2       2.347410
Name: data2, dtype: float64

### **Grouping with Dictionaries and Series**

In [156]:
# Example DataFrame 
# Setting the seed number to ensure reproducibility
rng= np.random.default_rng(seed=12345)
people= pd.DataFrame(rng.standard_normal((5,5)), columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steve', 'Wanda', 'Jill', 'Trey'])
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,0.968497,-0.759387,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [157]:
# Add a few NA values 
# Using loc() funtion -> label-based indexing
people.loc['Wanda', ['b', 'c']]= np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,NaN,NaN,0.902198,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [158]:
# Add a few NA values
# using iloc; "i" -> integer basis
people.iloc[2:3, [3]] = np.nan
people

,a,b,c,d,e
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863
Wanda,2.347410,NaN,NaN,NaN,-0.466953
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189


In [159]:
# Group correspondence for the columns
# A dictionary
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange', 'g' : 'violet'}

In [160]:
# Aggregate -> first we do a groupby, next we did something on the groupby obj
# T -> i want to do a transpose
# without transpose; blue and red is the row indeces
people.T.groupby(mapping).sum().T

,blue,red
Joe,-1.129835,-0.235440
Steve,1.009951,-4.061540
Wanda,0.000000,1.880456
Jill,-0.680811,2.127134
Trey,-0.718663,0.864410


In [161]:
# Assign new columns to a DataFrame
# Challenge: add the column f and g
rng = np.random.default_rng(seed=12345)
people = people.assign(f = rng.standard_normal((5, 1)),
                       g = rng.standard_normal((5, 1)))
people

,a,b,c,d,e,f,g
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863,1.263728,-1.367793
Wanda,2.347410,NaN,NaN,NaN,-0.466953,-0.870662,0.648893
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979,-0.259173,0.361058
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189,-0.075343,-1.952863


In [162]:
# Sum
people.T.groupby(mapping).sum()

,Joe,Steve,Wanda,Jill,Trey
blue,-1.129835,1.009951,0.000000,-0.680811,-0.718663
orange,-1.423825,1.263728,-0.870662,-0.259173,-0.075343
red,-0.235440,-4.061540,1.880456,2.127134,0.864410
violet,-0.740885,-1.367793,0.648893,0.361058,-1.952863


In [163]:
# Create Series
map_series= pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
g    violet
dtype: object

In [164]:
people.T.groupby(mapping).count()

,Joe,Steve,Wanda,Jill,Trey
blue,2,2,0,2,2
orange,1,1,1,1,1
red,3,3,2,3,3
violet,1,1,1,1,1


### **Grouping with Functions**

In [166]:
# Display content
people

,a,b,c,d,e,f,g
Joe,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
Steve,-0.740885,-1.367793,0.648893,0.361058,-1.952863,1.263728,-1.367793
Wanda,2.347410,NaN,NaN,NaN,-0.466953,-0.870662,0.648893
Jill,-0.060690,0.788844,-1.256668,0.575858,1.398979,-0.259173,0.361058
Trey,1.322298,-0.299699,0.902919,-1.621583,-0.158189,-0.075343,-1.952863


In [167]:
# Group length of the index letter
people.groupby(len).sum()

,a,b,c,d,e,f,g
3,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
4,1.261609,0.489146,-0.353749,-1.045725,1.240790,-0.334517,-1.591805
5,1.606525,-1.367793,0.648893,0.361058,-2.419816,0.393067,-0.718900


In [168]:
# Mixing functions with arrays, dictionaries or series
key_list= ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).max()

,,a,b,c,d,e,f,g
3,one,-1.423825,1.263728,-0.870662,-0.259173,-0.075343,-1.423825,-0.740885
4,two,1.322298,0.788844,0.902919,0.575858,1.398979,-0.075343,0.361058
5,one,2.347410,-1.367793,0.648893,0.361058,-0.466953,1.263728,0.648893


### **Grouping by Index Levels**

In [170]:
# A multiindex column
columns= pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1,3,5,1,3]],
                                   names= ['city', 'tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['city', 'tenor'])

In [186]:
# An example
rng= np.random.default_rng(seed=12345)
hier_df= pd.DataFrame(rng.standard_normal((4,5)), columns= columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -1.423825  1.263728 -0.870662 -0.259173 -0.075343
1     -0.740885 -1.367793  0.648893  0.361058 -1.952863
2      2.347410  0.968497 -0.759387  0.902198 -0.466953
3     -0.060690  0.788844 -1.256668  0.575858  1.398979

In [188]:
# Group by level
hier_df.T.groupby(level= 'tenor').count().T

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


## **13.2 Data Aggregation function**

In [192]:
# DataFrame from previous example
df

,key1,key2,data1,data2
0,a,1,-1.423825,0.648893
1,a,2,1.263728,0.361058
2,None,1,-0.870662,-1.952863
3,b,2,-0.259173,2.347410
4,b,1,-0.075343,0.968497
5,a,<NA>,-0.740885,-0.759387
6,None,1,-1.367793,0.902198


In [194]:
# Select the smallest requested number
# nsmallest(2) is the aggregation function
grouped= df.groupby('key1')
grouped['data1'].nsmallest(3) # nak 3 baris yg ada, start from smallest-largest

key1   
a     0   -1.423825
      5   -0.740885
      1    1.263728
b     3   -0.259173
      4   -0.075343
Name: data1, dtype: float64

In [197]:
# Define own aggregation function
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [198]:
# agg=aggregate
# apply the function to the dataframe
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.687553,1.408280
b,1,0.183830,1.378913


In [199]:
# Another way
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0 -0.300327  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0 -0.167258  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.261422  1.263728   3.0  0.083521  0.744032 -0.759387 -0.199165   
b    -0.121301 -0.075343   2.0  1.657953  0.975039  0.968497  1.313225   

                                    
           50%       75%       max  
key1                                
a     0.361058  0.504975  0.648893  
b     1.657953  2.002681  2.347410  

[2 rows x 24 columns]

### **Column-Wise and Multiple Function Application**

In [201]:
# Get the tipping dataset from github
# Download here: https://bit.ly/3VyE0vP
tips= pd.read_csv('https://bit.ly/3VyE0vP')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [202]:
tips['tip_pct']= tips['tip']/ tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [203]:
# Group the tips by day and smoker
grouped= tips.groupby(['day', 'smoker'])

In [204]:
# get descriptive statistics
grouped_pct= grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [205]:
# Getting back a DataFrame
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [206]:
# Another example
grouped_pct.agg([('Ave', 'mean'), ('Stdev', 'std')])

Ave     Stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [207]:
grouped.describe()

total_bill                                                        \
                 count       mean        std    min     25%     50%      75%   
day  smoker                                                                    
Fri  No            4.0  18.420000   5.059282  12.46  15.100  19.235  22.5550   
     Yes          15.0  16.813333   9.086388   5.75  11.690  13.420  18.6650   
Sat  No           45.0  19.661778   8.939181   7.25  14.730  17.820  20.6500   
     Yes          42.0  21.276667  10.069138   3.07  13.405  20.390  26.7925   
Sun  No           57.0  20.506667   8.130189   8.77  14.780  18.430  25.0000   
     Yes          19.0  24.120000  10.442511   7.25  17.165  23.100  32.3750   
Thur No           45.0  17.113111   7.721728   7.51  11.690  15.950  20.2700   
     Yes          17.0  19.190588   8.355149  10.34  13.510  16.470  19.8100   

                     tip            ...  size      tip_pct            \
               max count      mean  ...   75%  max   count      mean   
day  smoker                         ...                                
Fri  No      22.75   4.0  2.812500  ...  2.25  3.0     4.0  0.151650   
     Yes     40.17  15.0  2.714000  ...  2.00  4.0    15.0  0.174783   
Sat  No      48.33  45.0  3.102889  ...  3.00  4.0    45.0  0.158048   
     Yes     50.81  42.0  2.875476  ...  3.00  5.0    42.0  0.147906   
Sun  No      48.17  57.0  3.167895  ...  4.00  6.0    57.0  0.160113   
     Yes     45.35  19.0  3.516842  ...  3.00  5.0    19.0  0.187250   
Thur No      41.19  45.0  2.673778  ...  2.00  6.0    45.0  0.160298   
     Yes     43.11  17.0  3.030000  ...  2.00  4.0    17.0  0.163863   

                                                                         
                  std       min       25%       50%       75%       max  
day  smoker                                                              
Fri  No      0.028123  0.120385  0.137239  0.149241  0.163652  0.187735  
     Yes     0.051293  0.103555  0.133739  0.173913  0.209240  0.263480  
Sat  No      0.039767  0.056797  0.136240  0.150152  0.183915  0.291990  
     Yes     0.061375  0.035638  0.091797  0.153624  0.190502  0.325733  
Sun  No      0.042347  0.059447  0.139780  0.161665  0.185185  0.252672  
     Yes     0.154134  0.065660  0.097723  0.138122  0.215325  0.710345  
Thur No      0.038774  0.072961  0.137741  0.153492  0.184843  0.266312  
     Yes     0.039389  0.090014  0.148038  0.153846  0.194837  0.241255  

[8 rows x 32 columns]

In [209]:
# Specify funct to apply aggregate
functions= ['count', 'mean', 'max', 'min', 'median']
result= grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                                         total_bill  \
              count      mean       max       min    median      count   
day  smoker                                                              
Fri  No           4  0.151650  0.187735  0.120385  0.149241          4   
     Yes         15  0.174783  0.263480  0.103555  0.173913         15   
Sat  No          45  0.158048  0.291990  0.056797  0.150152         45   
     Yes         42  0.147906  0.325733  0.035638  0.153624         42   
Sun  No          57  0.160113  0.252672  0.059447  0.161665         57   
     Yes         19  0.187250  0.710345  0.065660  0.138122         19   
Thur No          45  0.160298  0.266312  0.072961  0.153492         45   
     Yes         17  0.163863  0.241255  0.090014  0.153846         17   

                                              
                  mean    max    min  median  
day  smoker                                   
Fri  No      18.420000  22.75  12.46  19.235  
     Yes     16.813333  40.17   5.75  13.420  
Sat  No      19.661778  48.33   7.25  17.820  
     Yes     21.276667  50.81   3.07  20.390  
Sun  No      20.506667  48.17   8.77  18.430  
     Yes     24.120000  45.35   7.25  23.100  
Thur No      17.113111  41.19   7.51  15.950  
     Yes     19.190588  43.11  10.34  16.470

In [210]:
# Passing a list of tuples
# tuple -> '()'
ftuples= [('AVERAGE', 'mean'), ('VARIANCE', 'var')]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct           total_bill            
              AVERAGE  VARIANCE    AVERAGE    VARIANCE
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [212]:
# Apply different functions using a dictionary
# dictionary -> {key-value pair}
grouped.agg({'tip':'max', 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

## **13.3 Apply: General split-apply-combine**

In [214]:
# Define a function to select the top five tip_pct values by group 
def top(df, n=5, column= 'tip_pct'):
    return df.sort_values(column, ascending= False)[:n]

In [215]:
# Get top 6
top(tips,n=8)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
149,7.51,2.00,No,Thur,Lunch,2,0.266312
93,16.32,4.30,Yes,Fri,Dinner,2,0.263480


In [216]:
# Groupby smoker
tips.groupby('smoker').apply(top, include_groups=False)

total_bill   tip   day    time  size   tip_pct
smoker                                                    
No     232       11.61  3.39   Sat  Dinner     2  0.291990
       149        7.51  2.00  Thur   Lunch     2  0.266312
       51        10.29  2.60   Sun  Dinner     2  0.252672
       185       20.69  5.00   Sun  Dinner     5  0.241663
       88        24.71  5.85  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15   Sun  Dinner     2  0.710345
       178        9.60  4.00   Sun  Dinner     2  0.416667
       67         3.07  1.00   Sat  Dinner     1  0.325733
       183       23.17  6.50   Sun  Dinner     4  0.280535
       109       14.31  4.00   Sat  Dinner     2  0.279525

In [218]:
# Pass additional argument after the function
# include_groups= False exclude 'smoker' & 'day' from the data that top() operates
tips.groupby(['smoker', 'day']).apply(top, n=3, column= 'total_bill', include_groups=False)

total_bill    tip    time  size   tip_pct
smoker day                                                
No     Fri  94        22.75   3.25  Dinner     2  0.142857
            91        22.49   3.50  Dinner     2  0.155625
            223       15.98   3.00   Lunch     3  0.187735
       Sat  212       48.33   9.00  Dinner     4  0.186220
            59        48.27   6.73  Dinner     4  0.139424
            23        39.42   7.58  Dinner     4  0.192288
       Sun  156       48.17   5.00  Dinner     6  0.103799
            112       38.07   4.00  Dinner     3  0.105070
            11        35.26   5.00  Dinner     4  0.141804
       Thur 142       41.19   5.00   Lunch     5  0.121389
            85        34.83   5.17   Lunch     4  0.148435
            141       34.30   6.70   Lunch     6  0.195335
Yes    Fri  95        40.17   4.73  Dinner     4  0.117750
            90        28.97   3.00  Dinner     2  0.103555
            96        27.28   4.00  Dinner     2  0.146628
       Sat  170       50.81  10.00  Dinner     3  0.196812
            102       44.30   2.50  Dinner     3  0.056433
            207       38.73   3.00  Dinner     4  0.077459
       Sun  182       45.35   3.50  Dinner     3  0.077178
            184       40.55   3.00  Dinner     2  0.073983
            180       34.65   3.68  Dinner     4  0.106205
       Thur 197       43.11   5.00   Lunch     4  0.115982
            83        32.68   5.00   Lunch     2  0.152999
            192       28.44   2.56   Lunch     2  0.090014

In [220]:
# Earlier example: Wide format
# describe() -> quick way of getting statistic data
result= tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [221]:
# Unstack: long format
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### **Suppressing the Group Keys**

In [226]:
tips.groupby('smoker').apply(top, include_groups=False)

total_bill   tip   day    time  size   tip_pct
smoker                                                    
No     232       11.61  3.39   Sat  Dinner     2  0.291990
       149        7.51  2.00  Thur   Lunch     2  0.266312
       51        10.29  2.60   Sun  Dinner     2  0.252672
       185       20.69  5.00   Sun  Dinner     5  0.241663
       88        24.71  5.85  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15   Sun  Dinner     2  0.710345
       178        9.60  4.00   Sun  Dinner     2  0.416667
       67         3.07  1.00   Sat  Dinner     1  0.325733
       183       23.17  6.50   Sun  Dinner     4  0.280535
       109       14.31  4.00   Sat  Dinner     2  0.279525

### **Quantile and Bucket Analysis**

In [228]:
# Another example
rng= np.random.default_rng(seed= 12345)
frame= pd.DataFrame({'data1': rng.standard_normal(1000),
                     'data2': rng.standard_normal(1000)})
frame.head()

,data1,data2
0,-1.423825,-1.176622
1,1.263728,1.205009
2,-0.870662,0.717879
3,-0.259173,-2.686273
4,-0.075343,-1.865260


In [229]:
# Split into quartile
# '(' -> parenthesis is exclusive / not included
# ']' -> bracket is inclusive / it is included
quartiles= pd.cut(frame['data1'],4)
quartiles.head()

0    (-1.537, 0.0459]
1     (0.0459, 1.629]
2    (-1.537, 0.0459]
3    (-1.537, 0.0459]
4    (-1.537, 0.0459]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.126, -1.537] < (-1.537, 0.0459] < (0.0459, 1.629] < (1.629, 3.211]]

In [233]:
# Passing a function
def get_stats(group):
    return pd.DataFrame(
        {'min':group.min(), 'max':group.max(),
         'count':group.count(), 'mean':group.mean()}
    )

In [236]:
# Create a groupby intermediate file
grouped= frame.groupby(quartiles, observed=False)

In [238]:
# Get the statistics
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.126, -1.537] data1 -3.119609 -1.538154     65 -2.019675
                 data2 -2.433466  2.040796     65  0.033381
(-1.537, 0.0459] data1 -1.523449  0.045684    443 -0.605092
                 data2 -2.830704  3.721098    443  0.020211
(0.0459, 1.629]  data1  0.048765  1.621952    438  0.678524
                 data2 -2.320815  3.461976    438  0.029518
(1.629, 3.211]   data1  1.651471  3.211418     54  2.044438
                 data2 -1.778229  2.530599     54  0.228316

In [240]:
# Using python internal function
grouped.agg(['min', 'max', 'count', 'mean'])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.126, -1.537] -3.119609 -1.538154    65 -2.019675 -2.433466  2.040796   
(-1.537, 0.0459] -1.523449  0.045684   443 -0.605092 -2.830704  3.721098   
(0.0459, 1.629]   0.048765  1.621952   438  0.678524 -2.320815  3.461976   
(1.629, 3.211]    1.651471  3.211418    54  2.044438 -1.778229  2.530599   

                                  
                 count      mean  
data1                             
(-3.126, -1.537]    65  0.033381  
(-1.537, 0.0459]   443  0.020211  
(0.0459, 1.629]    438  0.029518  
(1.629, 3.211]      54  0.228316

In [242]:
# Display frame dataframe content
frame.head()

,data1,data2
0,-1.423825,-1.176622
1,1.263728,1.205009
2,-0.870662,0.717879
3,-0.259173,-2.686273
4,-0.075343,-1.865260


In [244]:
# Quartile cut
quartiles_samp= pd.qcut(frame['data1'],4, labels=False)
quartiles_samp.head()

0    0
1    3
2    0
3    1
4    1
Name: data1, dtype: int64

In [246]:
# get stats
grouped= frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.119609 -0.678494    250 -1.294347
      data2 -2.830704  3.041653    250  0.062349
1     data1 -0.673305  0.008009    250 -0.303875
      data2 -2.686273  3.721098    250 -0.016510
2     data1  0.018753  0.686183    250  0.343512
      data2 -2.320804  3.461976    250 -0.025011
3     data1  0.688282  3.211418    250  1.287744
      data2 -2.320815  3.145451    250  0.124697

In [248]:
# A data
rng= np.random.default_rng(seed=12345)
s= pd.Series(rng.standard_normal(6))
s

0   -1.423825
1    1.263728
2   -0.870662
3   -0.259173
4   -0.075343
5   -0.740885
dtype: float64

In [251]:
# Add in NA
# '::' -> Skip of 2 ''
s[::2] = np.nan
s

0         NaN
1    1.263728
2         NaN
3   -0.259173
4         NaN
5   -0.740885
dtype: float64

In [252]:
# Fill NA with mean
s.fillna(s.mean())

0    0.087890
1    1.263728
2    0.087890
3   -0.259173
4    0.087890
5   -0.740885
dtype: float64

In [254]:
# Another data
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East', 'East', 'East', 'East',
             'West', 'West', 'West', 'West']
data = pd.Series(rng.standard_normal(8), index=states)
data

Ohio         -1.367793
New York      0.648893
Vermont       0.361058
Florida      -1.952863
Oregon        2.347410
Nevada        0.968497
California   -0.759387
Idaho         0.902198
dtype: float64

In [255]:
# Transform the data
data[['Vermont', 'Nevada', 'Idaho']]= np.nan
data

Ohio         -1.367793
New York      0.648893
Vermont            NaN
Florida      -1.952863
Oregon        2.347410
Nevada             NaN
California   -0.759387
Idaho              NaN
dtype: float64

In [258]:
# Getting the size
# size() include NA
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [259]:
# Getting the count
# count() doesn't include NA
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [260]:
# Getting mean
data.groupby(group_key).mean()

East   -0.890588
West    0.794011
dtype: float64

In [261]:
# Fill NA with the group means
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio         -1.367793
      New York      0.648893
      Vermont      -0.890588
      Florida      -1.952863
West  Oregon        2.347410
      Nevada        0.794011
      California   -0.759387
      Idaho         0.794011
dtype: float64

In [262]:
# With predefined fill values
fill_values = {'East':0.5, 'West':-1}

def fill_func(group):
    return group.fillna(fill_values[group.name])

# Fill using pre-defined values
data.groupby(group_key).apply(fill_func)

East  Ohio         -1.367793
      New York      0.648893
      Vermont       0.500000
      Florida      -1.952863
West  Oregon        2.347410
      Nevada       -1.000000
      California   -0.759387
      Idaho        -1.000000
dtype: float64

# **13.4 Group Transforms and “Unwrapped” GroupBys**
- do away from creating the groupby intermediate file

In [265]:
# An example
df= pd.DataFrame({'key':['a', 'b', 'c']*4,
                  'value':np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [268]:
# Group means by key
# using phyton default mean function
g= df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [269]:
# Define my own mean function
def get_mean(group):
    return group.mean()

# Using transform
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [270]:
# Another way - this is faster using default function
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [271]:
# transform() maintain the DataFrame structure
def times_two(group):
    return group*2

g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [272]:
# Display of content
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [274]:
# A NORMALIZATION function
# Method 1- define function
def normalize(x):
    return (x-x.mean()) / x.std()

g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [275]:
# Method 2 - could be slower as well
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [276]:
# unwrapped group operations - doing arithmetic between the outputs of
# multiple GroupBy operations
# often much faster than a general apply function
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### **Performance evaluation in terms of speed**
using **`%timeit`** function

In [279]:
# Method 1 - apply
%timeit g.apply(normalize)

# Method 2 - transform {faster}
%timeit g.transform(normalize)

# Method 3 - unwrapped
%timeit (df['value'] - g.transform('mean')) / g.transform('std')

24.3 ms ± 8.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
19 ms ± 3.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.41 ms ± 765 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## **Non-Vectorized (Looping)**

In [282]:
%%timeit

# Using for loop -> using a lot of for looping
a = np.array([1, 2, 3, 4, 5])
b = np.array([6, 7, 8, 9, 10])
result = []

for i in range(len(a)):
    result.append(a[i] + b[i])

# Output: [7, 9, 11, 13, 15]

24.3 μs ± 2.48 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## **Vectorized**

In [284]:
%%timeit

# Array
a = np.array([1, 2, 3, 4, 5])
b = np.array([6, 7, 8, 9, 10])

result = a + b

# Output: [7, 9, 11, 13, 15]

23.1 μs ± 4.22 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


# **13.5 Pivot Tables and Cross-Tabulation**
- A ***pivot table*** is a ***data summarization*** tool
- **rotating** or **reshaping** data

In [288]:
# From previous tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [289]:
# Getting mean for size and tip_pct columns
tips.pivot_table(index=['time', 'day'], columns='smoker',
                 values=['tip_pct', 'size'])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [361]:
# Display content of tips
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [363]:
# Use aggfunc = len for count
tips.pivot_table(index=['time', 'smoker'], columns='day',
                 values='tip_pct', aggfunc=len)

day            Fri   Sat   Sun  Thur
time   smoker                       
Dinner No      3.0  45.0  57.0   1.0
       Yes     9.0  42.0  19.0   NaN
Lunch  No      1.0   NaN   NaN  44.0
       Yes     6.0   NaN   NaN  17.0

## **Cross-Tabulations: Crosstab**
A ***special pivot table*** to compute ***group frequencies***

In [368]:
from io import StringIO

In [370]:
# data consists of strings
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

In [372]:
# Create a table
data = pd.read_table(StringIO(data), sep='\s+')

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ADMIN 2021\AppData\Local\Temp\ipykernel_14264\2503342613.py:2: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_table(StringIO(data), sep='\s+')


In [374]:
# Display content
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [376]:
# Crosstab example 1 - find group frequencies
# summarize by nationality and handedness
pd.crosstab(data['Nationality'], data['Handedness'], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [378]:
# tipping dataset
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [380]:
# Crosstab example 2
pd.crosstab([tips['time'], tips['day']], tips['smoker'], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244